## **Scikit - BLAS**

In [ ]:
# Uso de Scikit-CUDA

!pip install pycuda
!pip install scikit-cuda

# Uso de Scikit-CUDA

# cuBLAS (level 1)
# Operacion: SAXPY  (y = ax+y)
#  S -> Precision simple (S, D, C, Z)
#  a in R
#  x in R^n
#  y in R^n


import pycuda.autoinit
from pycuda import gpuarray
import numpy as np
from skcuda import cublas
import time

if __name__ == '__main__':

    a = np.float32(7.2)
    x = np.random.rand(5) # Probar vectores mayores
    y = np.random.rand(5)

    start = time.time()

    # 1. Vectores a memoria de la GPU
    x_gpu = gpuarray.to_gpu(x)
    y_gpu = gpuarray.to_gpu(y)

    # 2. Iniciar contexto
    cublas_ctxt = cublas.cublasCreate()

    # 3. Operación SAXPY
    cublas.cublasSaxpy (cublas_ctxt,    # Contexto
                        x_gpu.size,     # Tamano
                        a,              # Valor de a
                        x_gpu.gpudata,  # Valor de x
                        1,              # stride entre elementos de x
                        y_gpu.gpudata,  # Valor de y
                        1)              # stride entre elementos de y

    # 4. Finalizar contexto
    cublas.cublasDestroy(cublas_ctxt)

    end = time.time()
    print("Time: %.6f" % (end - start))
    y = y_gpu.get()
    print("Resultado:", y)


In [ ]:
# Uso de Scikit-CUDA

!pip install pycuda
!pip install scikit-cuda

# Uso de Scikit-CUDA

# cuBLAS (level 2)
# Operacion: GEMV  (y = alpha Ax + beta y)
#  alpha, beta in R
#  A in R^m,n
#  x in R^n
#  y in R^m


import pycuda.autoinit
from pycuda import gpuarray
import numpy as np
from skcuda import cublas
import time

if __name__ == '__main__':

    # Parametros
    tras = cublas._CUBLAS_OP['N']   # Traspuesta de A
    m = 10                          # filas de A
    n = 100                         # Columnas de A
    alpha = 1                       
    beta  = 0
    lda = m                         # leading dimension de A (m)

    A = np.random.rand(m, n).astype('float32') # Column-major
    x = np.random.rand(n) # Probar vectores mayores
    y = np.zeros(m, dtype=np.float32)

    start = time.time()

    # 1. Vectores a memoria de la GPU
    A_gpu = gpuarray.to_gpu(A)
    x_gpu = gpuarray.to_gpu(x)
    y_gpu = gpuarray.to_gpu(y)


    # 2. Iniciar contexto
    cublas_ctxt = cublas.cublasCreate()

    # 3. Operación SAXPY
    cublas.cublasSgemv (cublas_ctxt,    # Contexto
                        tras,           # traspuesta?
                        m,
                        n,
                        alpha,
                        A_gpu.gpudata,
                        lda,
                        x_gpu.gpudata,  # Valor de x
                        1,              # stride elementos de x
                        beta,
                        y_gpu.gpudata,  # Valor de y
                        1)              # stide elementos de y

    # 4. Finalizar contexto
    cublas.cublasDestroy(cublas_ctxt)

    end = time.time()
    print("Time: %.6f" % (end - start))
    y = y_gpu.get()
    print("Resultado:", y)


In [ ]:
# Uso de Scikit-CUDA

!pip install pycuda
!pip install scikit-cuda

# cuBLAS (level 3)
# Operacion: GEMM (matriz por matriz)
#   C = alpha A B + beta C
#  S -> precision simple (S, D, C, Z)
#  alpha, beta in R
#  A in R^m,k
#  B in R^k,n
#  C in R^m,n

import pycuda.autoinit
from pycuda import gpuarray
import numpy as np
from skcuda import cublas
import time

if __name__ == '__main__':

    # Parametros
    trasA = cublas._CUBLAS_OP['N']
    trasB = cublas._CUBLAS_OP['N']
    m = 5000
    n = 10000
    k = 8000
    alpha = 1.6
    beta  = 2.7
    lda = m
    ldb = k
    ldc = m

    A = np.random.rand(m, k).astype('float32') # Column-major
    B = np.random.rand(k, n).astype('float32') # Column-major
    C = np.random.rand(m, n).astype('float32') # Column-major

    start = time.time()

    # 1. Vectores a memoria de la GPU
    A_gpu = gpuarray.to_gpu(A)
    B_gpu = gpuarray.to_gpu(B)
    C_gpu = gpuarray.to_gpu(C)

    # 2. Iniciar contexto
    cublas_ctxt = cublas.cublasCreate()

    # 3. Operación GEMM
    cublas.cublasSgemm (cublas_ctxt,    # Contexto
                        trasA,          # traspuesta de A?
                        trasB,          # traspuesta de B?
                        m,
                        n,
                        k,
                        alpha,
                        A_gpu.gpudata,
                        lda,
                        B_gpu.gpudata,
                        ldb,
                        beta,
                        C_gpu.gpudata,  
                        ldc)

    # 4. Finalizar contexto
    cublas.cublasDestroy(cublas_ctxt)

    C = C_gpu.get()
    end = time.time()
    print("Time: %.6f" % (end - start))
    # print("Resultado:", C)


In [ ]:
# Uso de Scikit-CUDA

!pip install pycuda
!pip install scikit-cuda

# cuFFT
#   1-D

import pycuda.autoinit
from pycuda import gpuarray
import numpy as np
from skcuda import fft
import time


if __name__ == '__main__':

    x = np.random.rand(10000).astype('float32')

    start = time.time()

    x_gpu = gpuarray.to_gpu(x)
    x_fft = gpuarray.empty_like(x_gpu, dtype=np.complex64)

    # Configuracion
    plan = fft.Plan(x_gpu.shape, np.float32, np.complex64)

    # FFT
    fft.fft(x_gpu,
            x_fft,
            plan)

    end = time.time()
    print("Time: %6f" % (end - start))
    # print("Resultado: ", x_fft.get())
